In [1]:
!pip uninstall -y tensorflow --quiet
!pip install ludwig
!pip install ludwig[llm]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 19.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 682.2/682.2 kB 49.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.8/80.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.2/519.2 kB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.2/280.2 kB 33.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.1/98.1 kB 15.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.0/232.0 kB 25.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 71.4 MB/s eta 0:

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 2.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 65.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 30.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.7/174.7 kB 23.4 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=90ed0c4cd1140d05ef77e689ccb07f6b960c69f220424d25620b088634dcfc65
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))

get_ipython().events.register('pre_run_cell', set_css)

In [3]:
import os
import getpass
import locale

def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

os.environ["HUGGING_FACE_HUB_TOKEN"] = getpass.getpass("Token:")
assert os.environ["HUGGING_FACE_HUB_TOKEN"]

Token:··········


In [4]:
import pandas as pd

df = pd.read_json("https://raw.githubusercontent.com/sahil280114/codealpaca/master/data/code_alpaca_20k.json")
df = df.loc[df["input"]=='']  # pick only data that does not require any input column value
df = df.drop(["input"], axis=1)
df = df.tail(n=500)  # tried with 100 and then moved back to 500

In [5]:
import yaml
import logging
from ludwig.api import LudwigModel

"""
Uncomment the below to experiment with zero-shot prompting of the model.

Note that you may need to restart the notebook and comment out this section again
if you wish to perform the fine-tuning below, as running the zero-shot prompting
will allocate some additional memory on the GPU that can lead to CUDA OOM issues.
"""

zero_shot_config = yaml.safe_load(
  """
  model_type: llm
  base_model: Siddharthvij10/MistralSharded2

  input_features:
    - name: instruction
      type: text

  output_features:
    - name: output
      type: text

  prompt:
    template: >-
      Below is an instruction that describes a task.Write a response that appropriately completes the request.

      ### Instruction: {instruction}

      ### Response:

  generation:
    temperature: 0.1 # Temperature is used to control the randomness of predictions.
    max_new_tokens: 512

  adapter:
    type: lora

  quantization:
    bits: 4

  preprocessing:
    global_max_sequence_length: 512
    split:
      type: random

  trainer:
    type: finetune
    epochs: 1
    batch_size: 1
    eval_batch_size: 2
    gradient_accumulation_steps: 16
    learning_rate: 0.0004
    learning_rate_scheduler:
      warmup_fraction: 0.03

  """
)

model = LudwigModel(config=zero_shot_config, logging_level=logging.INFO)
results = model.train(dataset=df)

config.json:   0%|          | 0.00/613 [00:00<?, ?B/s]

INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒════════════════════════╕
INFO:ludwig.utils.print_utils:│ EXPERIMENT DESCRIPTION │
INFO:ludwig.utils.print_utils:╘════════════════════════╛
INFO:ludwig.utils.print_utils:
INFO:ludwig.api:╒══════════════════╤═════════════════════════════════════════════════════════════════════════════════════════╕
│ Experiment name  │ api_experiment                                                                          │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Model name       │ run                                                                                     │
├──────────────────┼─────────────────────────────────────────────────────────────────────────────────────────┤
│ Output directory │ /content/results/api_experiment_run                                                     │
├──────────────────┼─────────────────────────────────────────────────────────────────

tokenizer_config.json:   0%|          | 0.00/941 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

...

INFO:ludwig.trainers.trainer:
Running evaluation for step: 350, epoch: 0


Evaluation test : 100%|██████████| 50/50 [00:29<00:00,  1.70it/s]

INFO:ludwig.trainers.trainer:Evaluation took 42.7160s

INFO:ludwig.utils.metrics_printed_table:╒═══════════════════════╤════════════╤══════════════╤════════════╕
│                       │      train │   validation │       test │
╞═══════════════════════╪════════════╪══════════════╪════════════╡
│ bleu                  │     0.1898 │       0.1757 │     0.1800 │
├───────────────────────┼────────────┼──────────────┼────────────┤
│ char_error_rate       │     1.0758 │       1.1896 │     1.3536 │
├───────────────────────┼────────────┼──────────────┼────────────┤
│ loss                  │     0.8823 │       0.6134 │     0.7247 │
├───────────────────────┼────────────┼──────────────┼────────────┤
│ next_token_perplexity │ 15181.1406 │   14806.8838 │ 15152.5928 │
├───────────────────────┼────────────┼──────────────┼────────────┤
│ perplexity            │ 31850.9082 │   31530.5820 │ 31609.4004 │
├───────────────────────┼────────────┼──────────────┼────────────┤
│ rouge1_fmeasure       │     0.47


Training: 100%|██████████| 350/350 [04:31<00:00,  1.29it/s, loss=0.0483]

INFO:ludwig.utils.print_utils:
INFO:ludwig.utils.print_utils:╒═════════════════╕
INFO:ludwig.utils.print_utils:│ TRAINING REPORT │
INFO:ludwig.utils.print_utils:╘═════════════════╛
INFO:ludwig.utils.print_utils:
INFO:ludwig.api:╒══════════════════════════════╤═══════════════════╕
│ Validation feature           │ output            │
├──────────────────────────────┼───────────────────┤
│ Validation metric            │ loss              │
├──────────────────────────────┼───────────────────┤
│ Best model step              │ 350               │
├──────────────────────────────┼───────────────────┤
│ Best model epoch             │ 1                 │
├──────────────────────────────┼───────────────────┤
│ Best model's validation loss │ 0.613406240940094 │
├──────────────────────────────┼───────────────────┤
│ Best model's test loss       │ 0.724659264087677 │
╘══════════════════════════════╧═══════════════════╛
INFO:ludwig.api:
Finished: api_experiment_run
INFO:ludwig.api:Saved to: /content/re

In [8]:
df_inference = pd.read_json("https://raw.githubusercontent.com/sahil280114/codealpaca/master/data/code_alpaca_20k.json")

df_inference = df_inference.loc[df_inference["input"]=='']  # pick only data that does not require any input column value
df_inference = df_inference.drop(["input"], axis=1)

df_inference = df_inference.head(n=10)
df_inference

,instruction,output
0,Create an array of length 5 which contains all...,"arr = [2, 4, 6, 8, 10]"
1,Formulate an equation to calculate the height ...,Height of triangle = opposite side length * si...
3,Create an array of length 15 containing number...,"arr = [3, 6, 9, 12, 15, 18, 21, 24, 27, 30, 33..."
5,Create a nested loop to print every combinatio...,for i in range(10):\n for j in range(10):\n...
8,Write a class to represent a 2D point with x a...,"class Point:\n def __init__(self, x, y):\n ..."
10,Write a function that checks if a give number ...,def is_even(n):\n if n%2 == 0:\n ret...
11,Write code to create a list of all prime numbe...,def find_primes(n): \n prime_list = [2] \n ...
13,Write code to find the sum of all numbers betw...,"sum = 0\nfor i in range(1,11):\n sum += i\n..."
14,Create a function to calculate the area of a g...,def area_circle(radius):\n return 3.14 * (r...
16,Develop a classification algorithm in Python t...,import pandas as pd\nimport numpy as np\n\n# D...


In [9]:
predictions = model.predict(df_inference)[0]

INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of Siddharthvij10/MistralSharded2 tokenizer
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Prediction:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:ludwig.models.llm:Decoded text inputs for the first example in batch: below is an instruction that describes a task.write a response that appropriately completes the request.
### instruction: create an array of length 5 which contains all even numbers between 1 and 10.
### response:
INFO:ludwig.models.llm:Decoded generated output for the first example in batch: below is an instruction that describes a task.write a response that appropriately completes the request.
### instruction: create an array of length 5 which contains all even numbers between 1 and 10.
### response: [2, 4, 6, 8, 10]
INFO:ludwig.models.llm:Decoded text inputs for the first example in batch: below is an instruction that describes a task.write a response that appropriately completes the request.
### instruction: formulate an equation to calculate the height of a triangle given the angle, side lengths and opposite side length.
### response:
INFO:ludwig.models.llm:Decoded generated output for the first example in 

Prediction: 100%|██████████| 1/1 [00:53<00:00, 53.24s/it]


INFO:ludwig.utils.tokenizers:Loaded HuggingFace implementation of Siddharthvij10/MistralSharded2 tokenizer
/usr/local/lib/python3.10/dist-packages/ludwig/features/feature_utils.py:102: RuntimeWarning: divide by zero encountered in log
  return np.sum(np.log(sequence_probabilities))


In [12]:
!ludwig upload hf_hub --repo_id siddharthvij10/ludwig_finetuned --model_path /content/results/api_experiment_run

adapter_model.bin: 100% 13.7M/13.7M [00:02<00:00, 5.21MB/s]
Model uploaded to `https://huggingface.co/siddharthvij10/ludwig_finetuned/tree/main/` with repository name `siddharthvij10/ludwig_finetuned`
